In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl   
import matplotlib.pyplot as plt

**business objective**

to

**Data Cleansing**

Dun use: URL / ID / Name  

Drop : Average user rating (drop if NA)  

Subtitle (0,1-  not exist / exist) 

In-app purchase (0,1,2,3,5, --- number of in-app purchase options 

Language (0,1,2,3,5, --- number of language  options  

User rating count( varies too much, need logging  for predictive modelling )  

Age rating (later convert to 4+ -> 1... for model prediction) 

Developer (drop or find high rating developers) 

Description+ Icon URL   (leave it there for now) 

Size(in kb) 

Primary genre (categorize it) 

Genre (0/1 for each category ) <- under game strategy ge: 0 is no, 1 is have 

In [2]:
#import dataset
games = pd.read_csv('appstore_games.csv')
games.columns = games.columns.str.replace(' ', '')

games.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17007 entries, 0 to 17006
Data columns (total 18 columns):
URL                          17007 non-null object
ID                           17007 non-null int64
Name                         17007 non-null object
Subtitle                     5261 non-null object
IconURL                      17007 non-null object
AverageUserRating            7561 non-null float64
UserRatingCount              7561 non-null float64
Price                        16983 non-null float64
In-appPurchases              7683 non-null object
Description                  17007 non-null object
Developer                    17007 non-null object
AgeRating                    17007 non-null object
Languages                    16947 non-null object
Size                         17006 non-null float64
PrimaryGenre                 17007 non-null object
Genres                       17007 non-null object
OriginalReleaseDate          17007 non-null object
CurrentVersionReleaseDate

In [3]:
#Data cleansing

games['Languages'].fillna(games['Languages'].value_counts().index[0],inplace= True)
games['Size'].fillna(games['Size'].mean(),inplace=True)
games['Price'].fillna(games['Price'].value_counts().index[0],inplace= True)
games['UserRatingCount'].fillna(0,inplace=True)
games['AverageUserRating'].fillna(0,inplace=True)



In [4]:
games.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17007 entries, 0 to 17006
Data columns (total 18 columns):
URL                          17007 non-null object
ID                           17007 non-null int64
Name                         17007 non-null object
Subtitle                     5261 non-null object
IconURL                      17007 non-null object
AverageUserRating            17007 non-null float64
UserRatingCount              17007 non-null float64
Price                        17007 non-null float64
In-appPurchases              7683 non-null object
Description                  17007 non-null object
Developer                    17007 non-null object
AgeRating                    17007 non-null object
Languages                    17007 non-null object
Size                         17007 non-null float64
PrimaryGenre                 17007 non-null object
Genres                       17007 non-null object
OriginalReleaseDate          17007 non-null object
CurrentVersionReleaseDa

**data validation**

In [5]:
import pandas.util.testing as tm 
import seaborn as sns
import matplotlib.pyplot as plt

**Data Processing**

In [6]:
# drop non-useful attribute
games.drop(['IconURL'],axis=1,inplace=True)

In [7]:
# import encoder
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,MultiLabelBinarizer,normalize
from sklearn import preprocessing

In [8]:
ohe=OneHotEncoder()
le=LabelEncoder()
mlb = MultiLabelBinarizer()

In [9]:
# not_empty(1) empty(0) (Subtitle,In-appPurchases)
games['Subtitle'] = games['Subtitle'].apply(lambda x: 0 if pd.isnull(x) else 1)
games['In-appPurchases'] = games['In-appPurchases'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [10]:
#LabelEncoder (Price, AgeRating, PrimaryGenre)
games.Price= le.fit_transform(games.Price)
games.AgeRating= le.fit_transform(games.AgeRating)
games.PrimaryGenre= le.fit_transform(games.PrimaryGenre)
games.AverageUserRating= le.fit_transform(games.AverageUserRating)

In [11]:
#get_dummies (Languages, Genres)
Genres_dummies = games['Genres'].str.get_dummies(sep=', ').rename(lambda x: 'Genres_' + x, axis='columns')
games=pd.concat([games, Genres_dummies], axis=1)
Languages_dummies = games['Languages'].str.get_dummies(sep=', ').rename(lambda x: 'Languages_' + x, axis='columns')
games=pd.concat([games, Languages_dummies], axis=1)
games.drop(['Genres'],axis=1,inplace=True)
games.drop(['Languages'],axis=1,inplace=True)

In [12]:
#Frequency encoding (Developer)      
# size of each category
encoding = games.groupby('Developer').size()
# get frequency of each category
encoding = encoding*1000/len(games)
games['Developer_frequency'] = games['Developer'].map(encoding)
#Binning numerical values(Developer_frequency)
array = np.array([0,1,2])
games['Developer_frequency'] = array.searchsorted(games['Developer_frequency'])



In [13]:
# Binning numerical values and categorize (UserRatingCount, Size)       
games['UserRatingCount']=pd.qcut(games['UserRatingCount'].rank(method='first'),q=5)
games['UserRatingCount'] = le.fit_transform(games['UserRatingCount'])
games['Size']=pd.qcut(games['Size'],q=5)
games['Size']= le.fit_transform(games['Size'])

In [14]:
#extract year from both date columns (OriginalReleaseDate,CurrentVersionReleaseDate)
games["Original_Release_Year"] = ""
games['Original_Release_Year'] = 2020 - pd.DatetimeIndex(games['OriginalReleaseDate']).year
games.drop(['OriginalReleaseDate'],axis=1,inplace=True)
games["Current_Release_Year"] = ""
games['Current_Release_Year'] = 2020 - pd.DatetimeIndex(games['CurrentVersionReleaseDate']).year
games.drop(['CurrentVersionReleaseDate'],axis=1,inplace=True)

In [15]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
games.head(6)

,URL,ID,Name,Subtitle,AverageUserRating,UserRatingCount,Price,In-appPurchases,Description,Developer,AgeRating,Size,PrimaryGenre,Genres_Action,Genres_Adventure,Genres_Animals & Nature,Genres_Art,Genres_Board,Genres_Books,Genres_Business,Genres_Card,Genres_Casino,Genres_Casual,Genres_Comics & Cartoons,Genres_Education,Genres_Emoji & Expressions,Genres_Entertainment,Genres_Family,Genres_Finance,Genres_Food & Drink,Genres_Games,Genres_Gaming,Genres_Health & Fitness,Genres_Kids & Family,Genres_Lifestyle,Genres_Magazines & Newspapers,Genres_Medical,Genres_Music,Genres_Navigation,Genres_News,Genres_People,Genres_Photo & Video,Genres_Places & Objects,Genres_Productivity,Genres_Puzzle,Genres_Racing,Genres_Reference,Genres_Role Playing,Genres_Shopping,Genres_Simulation,Genres_Social Networking,Genres_Sports,Genres_Sports & Activities,Genres_Stickers,Genres_Strategy,Genres_Travel,Genres_Trivia,Genres_Utilities,Genres_Weather,Genres_Word,Languages_AF,Languages_AM,Languages_AR,Languages_AS,Languages_AY,Languages_AZ,Languages_BE,Languages_BG,Languages_BN,Languages_BO,Languages_BR,Languages_BS,Languages_CA,Languages_CS,Languages_CY,Languages_DA,Languages_DE,Languages_DZ,Languages_EL,Languages_EN,Languages_EO,Languages_ES,Languages_ET,Languages_EU,Languages_FA,Languages_FI,Languages_FO,Languages_FR,Languages_GA,Languages_GD,Languages_GL,Languages_GN,Languages_GU,Languages_GV,Languages_HE,Languages_HI,Languages_HR,Languages_HU,Languages_HY,Languages_ID,Languages_IS,Languages_IT,Languages_IU,Languages_JA,Languages_JV,Languages_KA,Languages_KK,Languages_KL,Languages_KM,Languages_KN,Languages_KO,Languages_KR,Languages_KS,Languages_KU,Languages_KY,Languages_LA,Languages_LO,Languages_LT,Languages_LV,Languages_MG,Languages_MI,Languages_MK,Languages_ML,Languages_MN,Languages_MR,Languages_MS,Languages_MT,Languages_MY,Languages_NB,Languages_NE,Languages_NL,Languages_NN,Languages_NO,Languages_OM,Languages_OR,Languages_PA,Languages_PL,Languages_PS,Languages_PT,Languages_QU,Languages_RN,Languages_RO,Languages_RU,Languages_RW,Languages_SA,Languages_SD,Languages_SE,Languages_SI,Languages_SK,Languages_SL,Languages_SO,Languages_SQ,Languages_SR,Languages_SU,Languages_SV,Languages_SW,Languages_TA,Languages_TE,Languages_TG,Languages_TH,Languages_TI,Languages_TK,Languages_TL,Languages_TO,Languages_TR,Languages_TT,Languages_UG,Languages_UK,Languages_UR,Languages_UZ,Languages_VI,Languages_XH,Languages_YI,Languages_ZH,Languages_ZU,Developer_frequency,Original_Release_Year,Current_Release_Year
0,https://apps.apple.com/us/app/sudoku/id284921427,284921427,Sudoku,0,7,4,3,0,"Join over 21,000,000 of our fans and download ...",Mighty Mighty Good Games,2,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,12,3
1,https://apps.apple.com/us/app/reversi/id284926400,284926400,Reversi,0,6,4,2,0,"The classic game of Reversi, also known as Oth...",Kiss The Machine,2,0,6,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,12,2
2,https://apps.apple.com/us/app/morocco/id284946595,284946595,Morocco,0,5,4,0,0,Play the classic strategy game Othello (also k...,Bayou Games,2,0,6,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,12,3
3,https://apps.apple.com/us/app/sudoku-free/i

In [16]:
target = ['AverageUserRating']
features = games.columns.values

X = games[['Subtitle','UserRatingCount', 'Price', 'In-appPurchases',
       'AgeRating', 'Size', 'PrimaryGenre', 'Genres_Action',
       'Genres_Adventure', 'Genres_Board', 'Genres_Books',
       'Genres_Business', 'Genres_Card', 'Genres_Casino', 'Genres_Casual',
       'Genres_Education', 'Genres_Emoji & Expressions',
       'Genres_Entertainment', 'Genres_Family', 'Genres_Finance',
       'Genres_Food & Drink', 'Genres_Games', 'Genres_Gaming',
       'Genres_Health & Fitness', 'Genres_Kids & Family',
       'Genres_Lifestyle', 'Genres_Magazines & Newspapers',
       'Genres_Medical', 'Genres_Music', 'Genres_Navigation',
       'Genres_News', 'Genres_Photo & Video', 'Genres_Productivity',
       'Genres_Puzzle', 'Genres_Racing', 'Genres_Reference',
       'Genres_Role Playing', 'Genres_Shopping', 'Genres_Simulation',
       'Genres_Social Networking', 'Genres_Sports', 'Genres_Stickers',
       'Genres_Strategy', 'Genres_Travel', 'Genres_Trivia',
       'Genres_Utilities', 'Genres_Weather', 'Genres_Word',
       'Languages_AF', 'Languages_AM', 'Languages_AR', 'Languages_AS',
       'Languages_AY', 'Languages_AZ', 'Languages_BE', 'Languages_BG',
       'Languages_BN', 'Languages_BO', 'Languages_BR', 'Languages_BS',
       'Languages_CA', 'Languages_CS', 'Languages_CY', 'Languages_DA',
       'Languages_DE', 'Languages_DZ', 'Languages_EL', 'Languages_EN',
       'Languages_EO', 'Languages_ES', 'Languages_ET', 'Languages_EU',
       'Languages_FA', 'Languages_FI', 'Languages_FO', 'Languages_FR',
       'Languages_GA', 'Languages_GD', 'Languages_GL', 'Languages_GN',
       'Languages_GU', 'Languages_GV', 'Languages_HE', 'Languages_HI',
       'Languages_HR', 'Languages_HU', 'Languages_HY', 'Languages_ID',
       'Languages_IS', 'Languages_IT', 'Languages_IU', 'Languages_JA',
       'Languages_JV', 'Languages_KA', 'Languages_KK', 'Languages_KL',
       'Languages_KM', 'Languages_KN', 'Languages_KO', 'Languages_KR',
       'Languages_KS', 'Languages_KU', 'Languages_KY', 'Languages_LA',
       'Languages_LO', 'Languages_LT', 'Languages_LV', 'Languages_MG',
       'Languages_MK', 'Languages_ML', 'Languages_MN', 'Languages_MR',
       'Languages_MS', 'Languages_MT', 'Languages_MY', 'Languages_NB',
       'Languages_NE', 'Languages_NL', 'Languages_NN', 'Languages_NO',
       'Languages_OM', 'Languages_OR', 'Languages_PA', 'Languages_PL',
       'Languages_PS', 'Languages_PT', 'Languages_QU', 'Languages_RN',
       'Languages_RO', 'Languages_RU', 'Languages_RW', 'Languages_SA',
       'Languages_SD', 'Languages_SE', 'Languages_SI', 'Languages_SK',
       'Languages_SL', 'Languages_SO', 'Languages_SQ', 'Languages_SR',
       'Languages_SU', 'Languages_SV', 'Languages_SW', 'Languages_TA',
       'Languages_TE', 'Languages_TG', 'Languages_TH', 'Languages_TI',
       'Languages_TK', 'Languages_TL', 'Languages_TO', 'Languages_TR',
       'Languages_TT', 'Languages_UG', 'Languages_UK', 'Languages_UR',
       'Languages_UZ', 'Languages_VI', 'Languages_YI', 'Languages_ZH',
       'Languages_ZU', 'Original_Release_Year',
       'Current_Release_Year']]

y = games[target]

In [17]:
#Import DecisionTreeClassifier and evaluation tools from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn import metrics

In [18]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40) # 70% training data & 30% testing 

In [19]:
model = DecisionTreeClassifier(max_depth=5, criterion='entropy')
model.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [20]:
#Predict test data and train data
pred_test = model.predict(X_test)
pred_train = model.predict(X_train)

In [21]:
#Evaluate result 
print("Accuracy from 30% test data:",accuracy_score(y_test, pred_test, normalize=True, sample_weight=None)*100,"%")

print("Accuracy from 70% train data:",accuracy_score(y_train, pred_train, normalize=True, sample_weight=None)*100,"%")

Accuracy from 30% test data: 73.93689986282578 %
Accuracy from 70% train data: 73.28629032258065 %


In [22]:
#import GridSearch
from sklearn.model_selection import GridSearchCV

In [23]:
# make an array of depths to choose from, say 1 to 40
depths = np.arange(1, 21)
depths

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [24]:
# define the number to try out for max leaf nodes 
num_leafs = [5, 10, 20, 50, 100, 200]
num_leafs

[5, 10, 20, 50, 100, 200]

In [25]:
try_grid = [{'max_depth':depths,
              'max_leaf_nodes':num_leafs}]

In [26]:
# define your Model using GridSearchCV
DTM = GridSearchCV(DecisionTreeClassifier(), param_grid=try_grid, cv=10)

In [27]:
DTM.fit(X,y)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid=[{'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20]),


In [28]:
# find the best parameters
DTM.best_params_

{'max_depth': 1, 'max_leaf_nodes': 5}

In [29]:
DTM.best_score_

0.7175280766743106

In [30]:
#Import cross validation 
from sklearn.model_selection import cross_val_score

In [31]:
#10-fold Cross validation score
score_cv = cross_val_score(model, X, y, cv=10)
score_cv

array([0.64633431, 0.71948357, 0.70287728, 0.72577804, 0.72486772,
       0.72470588, 0.71924662, 0.65783274, 0.71950501, 0.71361226])

In [32]:
print("Cross-validation score:",score_cv.mean()*100,"%")

Cross-validation score: 70.54243426122284 %


In [58]:
print(accuracy_score(y_test,y_pred))

0.01983252534156016


In [45]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5)

# Train the model using the training sets
knn_model.fit(X_train,y_train.values.ravel())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=24, p=2,
                     weights='uniform')

In [46]:
y_pred=knn_model.predict(X_test)

In [47]:
# Model Accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.7160493827160493


In [37]:
from sklearn.model_selection import RandomizedSearchCV

In [38]:
num_leaf_size = np.arange(1, 11)
num_leaf_size

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [39]:
num_n_neighbors = np.arange(1, 31)
num_n_neighbors

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [40]:
try_grid = {'leaf_size':num_leaf_size,
              'n_neighbors':num_n_neighbors}

In [41]:
# define your Model using RandomizedSearchCV
randomized_search = RandomizedSearchCV(estimator=KNeighborsClassifier(), param_distributions=try_grid, cv=10)

In [43]:
randomized_search.fit(X_train, y_train.values.ravel())

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=5,
                                                  p=2, weights='uniform'),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'leaf_size': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [44]:
randomized_search.best_params_

{'n_neighbors': 24, 'leaf_size': 8}

In [49]:
randomized_search.best_score_

0.7106014784946236